# Load emissivity data

In this example, you will lear how to load data from emissivity grid and do simple interpolation. As a first step, you will need to import innet and pyneb library. Pyneb is a tool to compute emission line emissivities.

In [1]:
import innate
import pyneb as pn

First, load the emissivity data in HDF5 format as follows:

In [2]:
data_file = '../data/emissivity_grids.nc'
emissivities = innate.DataSet.from_file(data_file)

Data set "O2_3726A_m" is missing:
Parametrisation formula ("eqn" key in dataset configuration).
Parametrisation coefficients ("eqn_coeffs" key in dataset configuration).
Data set "Ne3_3869A" is missing:
Parametrisation formula ("eqn" key in dataset configuration).
Parametrisation coefficients ("eqn_coeffs" key in dataset configuration).
Data set "Fe3_4658A" is missing:
Parametrisation formula ("eqn" key in dataset configuration).
Parametrisation coefficients ("eqn_coeffs" key in dataset configuration).
Data set "H1_4861A" is missing:
Parametrisation formula ("eqn" key in dataset configuration).
Parametrisation coefficients ("eqn_coeffs" key in dataset configuration).


As a next step, you will compute an approximated value for the emissivity of twice ionized oxygen (O$^{2+}$) for a wavelength 5007 $\mathrm{\mathring A}$ (500.7 nm), i.e., O3_5007A point, for temperature 12250 K and density 122 particles/cm$^3$ using Regular Grid Interpolator.

In [9]:
temp, den = 12250, 122
interp_value_innate = emissivities['O3_5007A'].approx.interp.rgi((temp, den)).eval()
print(f'Interpolated value for O3_5007A is: {interpolated_value}')

Interpolated value for O3_5007A is: [54144.39828319]


To confirm the accuracy of the interpolation, let's compare the interpolated value with the one obtained by RegularGridInterpolator from scipy library:

In [11]:
from scipy.interpolate import RegularGridInterpolator
import numpy as np

data = emissivities['O3_5007A'].data
x = list(emissivities['O3_5007A'].axes_range.values())[0]
y = list(emissivities['O3_5007A'].axes_range.values())[1]

xg, yg = np.meshgrid(x, y, indexing='ij', sparse=True)

interp = RegularGridInterpolator((x, y), data)

interpol_value_scipy = interp([12250, 122])

print(f'Interpolated value for O3_5007A from scipy is: {interpol_value_scipy}')

difference = interpol_value_scipy - interp_value_innate

print(f'Difference between the intepolations is: {difference}')


Interpolated value for O3_5007A from scipy is: [54144.39828319]
Difference between the intepolations is: [0.]


From this comparison, you can see that Innate automates the interpolation process and provides the same accuracy as methods implemented in other libraries.